In [ ]:
# Import google cloud library
from google.cloud import bigquery
from google.cloud import storage
from google.cloud import aiplatform
import pickle
import pandas as pd
import numpy as np

In [ ]:
# create new prediction data
new_test_data = pd.DataFrame({
    'humidity': [0.28,0.5],
    'weather': [1,3],
    'holiday': [0,0],
    'season': [1,3],
    'temperature': [0.7,0.7],
    'hour': [15,3],
    'dayofweek': [6,0],
    'month': [10,6],
    'year': [2011,2012],
})

new_test_data

In [ ]:
# Authenticate services account
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "/home/e2102ta/safira-001/sa-development.json"

In [ ]:
project_id = 'dti-ds'
dataset_id = 'safira_dataset_001'
table_id = 'X_test'
region = 'us-central1'
bucket_name = 'safira_gcs_001'
blob_name = 'data/X_test.csv'

model_name = 'bike_model.pkl'

In [ ]:
#load data from BQ
from google.cloud import bigquery
## using bigquery client 
# client = bigquery.Client(credentials=credentials,project=project_id)
client = bigquery.Client(project='dti-ds')

# query 
query_job = client.query(f"""select * from {dataset_id}.{table_id}""")
auto_cloud = query_job.result().to_dataframe()

#cleansing 
result = auto_cloud.drop(['int64_field_0'], axis = 1)
result

In [ ]:
# Load the model
with open('bike_model.pkl', 'rb') as f:
    loaded_model = pickle.load(f)

y_pred_cloud_new_data = loaded_model.predict(new_test_data)
y_pred_cloud_new_data

In [ ]:
new_test_cloud = new_test_data.copy()
new_test_cloud['Count Predict'] = y_pred_cloud_new_data # new column
new_test_cloud

In [ ]:
# loaded_model.predict(X_test.iloc[3:13])
y_pred_file_cloud = loaded_model.predict(auto_cloud)
y_pred_file_cloud[:13]

In [ ]:
result['Count Predict'] = y_pred_file_cloud # new column
result

In [ ]:
table_id = 'X_test_predicted'

# Construct a BigQuery client object.
client = bigquery.Client()

# Define the full table ID
table_full_id = f"{client.project}.{dataset_id}.{table_id}"

result.columns = ['_'.join(i.split(' ')) for i in result.columns] # make sure no empty space on columns' name

# Create the table
# table = bigquery.Table(table_full_id)
# table = client.create_table(table, exists_ok=True)
# print(f"Created table {table.project}.{table.dataset_id}.{table.table_id}")

# Load the DataFrame into the BigQuery table
job = client.load_table_from_dataframe(result, table_full_id) # your df predicted name, and table full id above

# Wait for the job to complete
job.result()
print(f"Loaded {job.output_rows} rows into {table_full_id}")